<a href="https://colab.research.google.com/github/Kathy-Xueqing-Wang/Exercises_TensorFlow-in-practice/blob/master/Week4_Practice2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This time we build a CNN model for Happy and Sad Faces binary classification. The fit_generator is used for this practice.

In [1]:
import tensorflow as tf
# import these libraries to help load the dataset
import os
import zipfile

In [2]:
os.getcwd()

'/content'

Unzip the data: note that the data will be deleted when the kernal stops... so maybe you cannot access the data.. I'll try to put the data somewhere on github.

In [3]:
path = f"{os.getcwd()}/happy-or-sad.zip"
zip_ref = zipfile.ZipFile(path, 'r')
zip_ref.extractall(f"{os.getcwd()}/h_or_s")
zip_ref.close()

Then we build the generator:

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1/255)

train_generator = train_datagen.flow_from_directory(
    '/content/h_or_s/',
    target_size = (300, 300),
    batch_size = 10,
    class_mode = 'binary'
)

Found 80 images belonging to 2 classes.


Now it is the callback class.

In [5]:
class MyCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs = {}):
    if(logs.get('acc')>0.999):
      print('\nReached 99.9% accuracy so cancelling training!')
      self.model.stop_training = True

All set! It is time to build the model.

In [6]:
callbacks = MyCallback()

model = tf.keras.models.Sequential([
                                    tf.keras.layers.Conv2D(16, (3, 3), input_shape = (300, 300, 3), activation= 'relu'),
                                    tf.keras.layers.MaxPooling2D((2, 2)),
                                    tf.keras.layers.Conv2D(32, (3, 3), activation= 'relu'),
                                    tf.keras.layers.MaxPooling2D((2, 2)),
                                    tf.keras.layers.Conv2D(64, (3, 3), activation= 'relu'),
                                    tf.keras.layers.MaxPooling2D((2, 2)),
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(128, activation='relu'),
                                    tf.keras.layers.Dense(1, activation='sigmoid')
])

from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer = RMSprop(lr = 0.001), loss = 'binary_crossentropy', metrics = 'acc')

In [7]:
model.fit_generator(
    train_generator, 
    steps_per_epoch = 8,
    epochs = 20,
    verbose = 1,
    callbacks = [callbacks]
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
8/8 [==============================] - 0s 29ms/step - loss: 2.0591 - acc: 0.5375
Epoch 2/20
8/8 [==============================] - 0s 32ms/step - loss: 0.5649 - acc: 0.7500
Epoch 3/20
8/8 [==============================] - 0s 28ms/step - loss: 0.2117 - acc: 0.9375
Epoch 4/20
8/8 [==============================] - 0s 28ms/step - loss: 0.5474 - acc: 0.7875
Epoch 5/20
8/8 [==============================] - 0s 28ms/step - loss: 0.0915 - acc: 0.9875
Epoch 6/20
8/8 [==============================] - 0s 29ms/step - loss: 0.0449 - acc: 0.9875
Epoch 7/20
8/8 [==============================] - 0s 29ms/step - loss: 0.0409 - acc: 0.9875
Epoch 8/20
7/8 [=========================>....] - ETA: 0s - loss: 0.0147 - acc: 1.0000
Reached 99.9% accuracy so cancelling training!
8/8 [==============================] - 0s 30ms/step - loss: 0.0130 - acc: 1.0000


Did that! So easy.